# Construir Dataset + Preprocesamiento PNL

In [107]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt 
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition  import LatentDirichletAllocation
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.stem import WordNetLemmatizer

In [108]:
# Actual dir
path = "dataset"
if path not in os.getcwd():
    os.chdir(os.getcwd()) 

path_actual =     os.getcwd()
print("Path actual: ",path_actual)
df = pd.read_csv(os.getcwd()+"/dataset/okcupid_profiles.csv")

Path actual:  /home/iciac/Escritorio/CursoIA/AprendizajeNoSupervisado


In [109]:
#Variables continuas
df.describe()

,age,height,income
count,59946.000000,59943.000000,59946.000000
mean,32.340290,68.295281,20033.222534
std,9.452779,3.994803,97346.192104
min,18.000000,1.000000,-1.000000
25%,26.000000,66.000000,-1.000000
50%,30.000000,68.000000,-1.000000
75%,37.000000,71.000000,-1.000000
max,110.000000,95.000000,1000000.000000


In [110]:
print("Valores Nulos \n-----------")
df.isnull().sum()

Valores Nulos 
-----------


age                0
status             0
sex                0
orientation        0
body_type       5296
diet           24395
drinks          2985
drugs          14080
education       6628
ethnicity       5680
height             3
income             0
job             8198
last_online        0
location           0
offspring      35561
pets           19921
religion       20226
sign           11056
smokes          5512
speaks            50
essay0          5488
essay1          7572
essay2          9638
essay3         11476
essay4         10537
essay5         10850
essay6         13771
essay7         12451
essay8         19225
essay9         12603
dtype: int64

## Prepocesamiento PNL

Existen nueve columnas que describen los gustos de los usuarios, las hemos juntado para tratar de extraer los temas principales.

In [111]:
essaysname = ["essay"+str(i) for i in range(0,10)]
profiles = df[essaysname].copy()

In [112]:
profiles.tail(5)

,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
59941,"vibrant, expressive, caring optimist. i love b...",the happiest times have been when life came to...,i make an outstanding osso bucco. i am also ve...,"i am told that people notice my smile, eyes an...",i am an avid movie watcher and follow the broa...,"my family, my dog, italy, words and music!",writing my book.,"running with my dog, finishing up the work wee...",i have a dream to sing at the alconquin in nyc...,you are seeking a long term connection of shar...
59942,i'm nick. i never know what to write about mys...,currently finishing school for film production...,"filmmaking, photography, graphic design, web d...","dude, i don't know.","movies: hook (the greatest adventure ever!), g...",iphone contact lenses headphones camera tv rem...,i do most of my thinking on the bus to/from wo...,"bringin' home bacon, or drinking and shakin'!",when i was 18 i got a tattoo of waldo somewher...,meh if you made it this far you might as well.
59943,"hello! i enjoy traveling, watching movies, and...","i'm a civil engineer, who enjoys helping the c...",- looking at things objectively - getting thin...,i'm quiet until i get used to the environment ...,"last book: ""game change"". movies: bourne serie...",- iphone - friends and family - internet - bay...,"aside from work, how to improve my home.",out enjoying friendly conversation over dinner.,please let me think about this more.,we have similar interests.
59944,"""all i have in this world are my balls and my ...","following my dreams... ""you got a dream... you...",listening,it used to be the hair until i mowed it off bu...,where to begin musically: right now i listen t...,"music, family, friends, a basketball, hoop, so...",what can i do to make someone chuckle....,what i would do on any other day. everydays a ...,i like walking around in other people's house ...,you are interested and interesting...
59945,"is it odd that having a little ""enemy"" status ...",i work with elderly people (psychotherapy and ...,i'm a great bullshitter. i don't know what it ...,"either that i am funny/sarcastic, or that i am...","i just read the help by kathryn stockett, sooo...",1. family & friends & other humans - interacti...,"sex, myself, other people, how amazing everyth...","out at happy hour with my friends, running int...",i wish i could cry like holly hunter in broadc...,"if you have a back-bone, an opinion, a sense o..."


In [113]:
# Sustituir valores nan por '' 
profiles.fillna(" ", inplace=True)

In [114]:
#Unificacion de todos los essays a una sola columna
essays = []
essays = profiles.essay0 + ' ' + profiles.essay1 + ' ' + profiles.essay2 + ' ' + profiles.essay3 + ' ' + profiles.essay4 + ' ' + profiles.essay5 + ' ' + profiles.essay6 + ' ' + profiles.essay7 + ' ' + profiles.essay8 + ' ' + profiles.essay9
profiles['all_essays'] = essays

if "essay1" in profiles.columns:
    profiles.drop(columns=essaysname, inplace=True)

if (profiles.isnull().sum().to_list() == []): 
    print("No hay nulos")
    
profiles 

,all_essays
0,about me: i would love to think that i was so...
1,i am a chef: this is what that means. 1. i am ...
2,"i'm not ashamed of much, but writing public te..."
3,i work in a library and go to school. . . read...
4,hey how's it going? currently vague on the pro...
...,...
59941,"vibrant, expressive, caring optimist. i love b..."
59942,i'm nick. i never know what to write about mys...
59943,"hello! i enjoy traveling, watching movies, and..."
59944,"""all i have in this world are my balls and my ..."


In [115]:
new_words = ['still','take','trying','great','go','little','live','good','big','years','love','right','back','mine','thing','things',
             'better','anything','really','able','half','made','way','make','someone','everything','kind','see','type','types','new',
             'more','much','lot','though','unless','probably','usually','looking','also','something','know','think','like','enjoy',
             'well','time','making','always','try','one','would','around','many','making','going','want','trying','open','long',
             'pretty','say','sometimes','getting','best','stuff','person','day','favorite','feel','old','smile','guy','life','living',
             'afluster', 'dfg', 'eleifend', 'emfoi', 'ewoe', 'faucibus', 'fgd', 'frakking', 'fsd', 'fsdf', 'ghtr', 'hilda', 'imm',
             'maecenas', 'mafoiawm', 'malesuada', 'moewaaoicmw', 'moiwa', 'mweiof', 'nn', 'oma', 'pharetra', 'praline', 'rg', 'rth',
             'saah', 'sdf', 'sfd', 'thous', 'torolf', 'utm', 'wahh', 'woi', 'wordsi', 'yuval','aoicmw', 'bhakti', 'bluetoonist', 'citation',
             'curabitur', 'dfgd', 'epr', 'gosto', 'gsdsd', 'hallie', 'inhabitable', 'lobortis', 'lorelai', 'lorry', 'lovebot', 'luctus',
             'mdmj', 'moewa', 'ner', 'pandit', 'prema', 'proident', 'realmente', 'rgd', 'sfsd', 'sgi', 'slats', 'sugalumps',
             'superintendent', 'teehehe', 'tramps', 'ustad', 'varius', 'vulputate', 'yew', 'annun', 'artsier', 'autobus',
             'congue', 'enfjs', 'eplariza', 'exercitation', 'fru', 'gatorar', 'gazelle', 'godhead', 'incididunt', 'infps',
             'jrpg', 'kenner', 'kielbasa', 'lipps', 'meep', 'messi', 'motherflippin', 'nfnc', 'nostrud', 'ralston', 'ratkins',
             'roiphe', 'ruffin', 'rutrum', 'saraswati', 'sifu', 'stefisbuff', 'strongbow', 'suscipit', 'tili', 'unst', 'walla', 'aliqua',
             'aliquip', 'andouille', 'apna', 'ashzra', 'auch', 'congruence', 'cupidatat', 'demisexual', 'dolore', 'ecoutez', 'elly', 'frnds',
             'fuckups', 'fuk', 'kladdkaka', 'klove', 'laboris', 'lov', 'lupus', 'melia', 'nao', 'omnicircus', 'paciocco', 'quidam', 'rere',
             'setters', 'sharpens', 'singe', 'sint', 'teksto', 'thaat', 'tribunal', 'waaahh', 'wawah', 'aglets', 'akashic', 'belladonna', 'boingy',
             'breatharian', 'buk', 'cafepress', 'cyclenator', 'ecopsychology', 'esse', 'excetera', 'fnordloki', 'fortran', 'gado', 'hoon',
             'horsey', 'krylon', 'lawton', 'noc', 'occitan', 'officia', 'orale', 'pawns', 'pwc', 'ql', 'reactivity', 'rutgers', 'sexanddesign',
             'slocks', 'thoe', 'toga', 'tudi', 'voluptate', 'wedged', 'zander','adipisicing', 'alota', 'arnall', 'arrr', 'babygirl',
             'bjrn', 'chanchito', 'consonant', 'dob', 'dunh', 'duval', 'eyeware', 'ftapon', 'gallbladder', 'guiltied', 'gust', 'haulin',
             'jetzt', 'kma', 'lili', 'matchy', 'melinda', 'moocow', 'muzix', 'mysore', 'oshun', 'porttitor', 'prendre', 'rnr', 'saige',
             'sattvic', 'tabor', 'tartan', 'teaspoons', 'unemotional','aanq', 'aku', 'alg', 'allister', 'aoi', 'debrief', 'estar', 'felling',
             'fw', 'guyana', 'hilbert', 'hornier', 'humin', 'jjc', 'joyfulparadox', 'larm', 'layne', 'livros', 'maj', 'mii', 'muziq', 'newsstand',
             'nullalux', 'oooooooooaaaaaaawwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwww',
             'peices', 'pitted', 'quelque', 'resently', 'rly', 'sleeeeeping', 'supplementation', 'topher', 'tristissima', 'viel', 'virgule', 'people','world', 'fun', 'work', 'working']

stopwords_customized = stopwords.words('english') + new_words + list(ENGLISH_STOP_WORDS)


In [116]:
def preprocess_text(text):
    review = re.sub(r'\W', ' ', text) #sentences[i]     # Sustituye los caracteres no alfanuméricos por espacio
    review = review.lower()                             # Conversion a minusculas
    review = re.sub(r'^br$', ' ', review)               # Los que empiecen por 'b' y terminen por 'r' se sustituyen por un espacio en blanco
    review = re.sub(r'\s+br\s+',' ',review)             # El patron (espacio en blanco o más, br, espacio en blanco o mas) se sustituye por un espacio en blanco
    review = re.sub(r'\s+[a-z]\s+', ' ',review)         # El patron (espacio en blanco o más, cualquier letra del alfabeto, espacio en blanco o más) se sustituye por un espacio en blanco
    review = re.sub(r'^b\s+', '', review)               # Comienzo de la cadena con 'b' seguido de uno o mas espacios en blanco lo elimina
    review = re.sub(r'\s+', ' ', review)                # Eliminar los espacios en blanco sobrantes incluidos en los pasos anteriores.
    review = re.sub('[^a-zA-Z]+', ' ', review)
    #corpus.append(review)r'\s+'
    return review

def tokenize(text):
    sentences = nltk.sent_tokenize(text)
    return sentences


In [117]:
fusion = []
for i in range(len(essays)):
    fusion.append(preprocess_text(essays[i]))
len(fusion)    

59946

In [118]:
#Convert a collection of text documents to a matrix of token counts
vectorizer = CountVectorizer(max_features = 20, min_df = 0.25, max_df = 0.85, stop_words = stopwords_customized ) #Convert a collection of text documents to a matrix of token counts
X = vectorizer.fit_transform(fusion).toarray()
X.shape

(59946, 10)

In [119]:
vectorizer.vocabulary_

{'reading': 9,
 'books': 0,
 'movies': 6,
 'music': 7,
 'food': 2,
 'friends': 3,
 'read': 8,
 'movie': 5,
 'family': 1,
 'home': 4}

In [120]:
# Mostrar las características.
features = vectorizer.get_feature_names()
print(features)

['books', 'family', 'food', 'friends', 'home', 'movie', 'movies', 'music', 'read', 'reading']


In [121]:
######LDA
# Obtener los temas.
n_topics = 4
lda = LatentDirichletAllocation(n_components=n_topics, random_state=123)
my_docs_topic = lda.fit_transform(X)  

In [122]:
my_docs_topic

array([[0.03689712, 0.32303481, 0.60387851, 0.03618957],
       [0.12777698, 0.12540162, 0.620254  , 0.1265674 ],
       [0.50949537, 0.27760066, 0.02624686, 0.18665711],
       ...,
       [0.48125781, 0.02014014, 0.34803398, 0.15056807],
       [0.61785289, 0.31298927, 0.0367009 , 0.03245695],
       [0.22491138, 0.55071749, 0.1153142 , 0.10905693]])

In [123]:
topic_composition = lda.components_

In [124]:
n_top = 4
for i in range(n_topics):
    topic_features = [features[idx] for idx in np.argsort(-topic_composition[i,:])]   # argsort() muestra el índice ordenado.
    topic_features_top = topic_features[0:n_top]
    if i == 0:
        topic_matrix = [topic_features_top]                    # list의 list 만들 준비!
    else:
        topic_matrix.append(topic_features_top) 

In [125]:
# Muestra las características principales de cada tema.
topic_matrix

[['friends', 'family', 'food', 'music'],
 ['read', 'reading', 'books', 'movies'],
 ['music', 'food', 'movies', 'books'],
 ['home', 'movie', 'friends', 'family']]

In [126]:
# En vista de las características principales, podemos nombrar los temas.
topic_names = ['Social','Book','Music','Movie']

In [127]:
# El tema más probable viene dado directamente por la salida de LDA.
n_docs = len(essays)
labels = []
for i in range(n_docs):
    topic_pick = np.argmax(my_docs_topic[i,:])
    #print("Document " + str(i+1) + " = " + topic_names[topic_pick])
    labels.append(topic_names[topic_pick])
    
#Nueva columna en pandas con las etiquetas
profiles['essays'] = labels    
if "all_essays" in profiles.columns.to_list():
    profiles.drop(columns="all_essays", inplace=True)
profiles

,essays
0,Music
1,Music
2,Social
3,Book
4,Music
...,...
59941,Movie
59942,Music
59943,Social
59944,Social


In [133]:
### Join by index with our dataset
df = pd.merge(df, profiles, left_index=True, right_index=True)
df.drop(columns = ["essay"+str(i) for i in range(0,10)], inplace=True)
df

## Construir Dataset


### Categoricas Obligatorias: Separar en datasets 
- Homosexual mujer (Bisexual mujer)
- Homosexual hombre (Bisuxeal hombre)
- Heterosexuales (Bisexual)

En todas podemos borrar sex menos en heterosexual


In [129]:
df["sex"].unique()

array(['m', 'f'], dtype=object)

In [130]:
df["orientation"].unique()

array(['straight', 'bisexual', 'gay'], dtype=object)

In [131]:
homosexual_f = df[("f" == dataset["sex"]) & (dataset["orientation"].isin(["bisexual", "gay"]))]
homosexual_h = df[("m" == dataset["sex"]) & (dataset["orientation"].isin(["bisexual", "gay"]))]
heterosexual = df[(dataset["orientation"].isin(["bisexual", "straight"]))]

homosexual_f.drop(columns = ["sex","orientation"], inplace = True)
homosexual_h.drop(columns = ["sex","orientation"], inplace = True)
heterosexual.drop(columns = ["orientation"], inplace = True)

homosexual_f.to_csv(path_actual+"/dataset/homosexualf.csv", index= False)
homosexual_h.to_csv(path_actual+"/dataset/homosexualh.csv", index= False)
heterosexual.to_csv(path_actual+"/dataset/heterosexual.csv", index= False)

/home/iciac/.local/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
